In [2]:
import argparse
import logging
import time
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer, ngrams_iterator
from torchtext.datasets import DATASETS
from torchtext.prototype.transforms import load_sp_model, PRETRAINED_SP_MODEL, SentencePieceTokenizer
from torchtext.utils import download_from_url
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torchviz import make_dot
from tqdm import tqdm

### Information
- torchtext repo: https://github.com/pytorch/text/tree/main/torchtext
- torchtext documentation: https://pytorch.org/text/stable/index.html

### Constants

In [3]:
DATASET = "WikiText2"
DATA_DIR = ".data"
DEVICE = "cpu"
LR = 4.0
BATCH_SIZE = 16
NUM_EPOCHS = 5
MIN_FREQUENCY = 5
PADDING_VALUE = 0
PADDING_IDX = PADDING_VALUE

# n-gram level
n = 3
# Hidden layer dimension
h = 100
# Word embedding dimension
m = 100

### Get the tokenizer

In [4]:
basic_english_tokenizer = get_tokenizer("basic_english")

In [5]:
basic_english_tokenizer("This is some text ...")

['this', 'is', 'some', 'text', '.', '.', '.']

In [6]:
# Needed later.
TOKENIZER = basic_english_tokenizer

### Get the data and get the vocabulary.

In [7]:
def yield_tokens(data_iter):
    for text in data_iter:
        yield TOKENIZER(text)

In [8]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")
VOCAB = build_vocab_from_iterator(yield_tokens(train_iter), min_freq = MIN_FREQUENCY, specials=['<unk>'])

# Set the default index to 1. Otherwise, VOCAB['unknownbigword'] will raise an Exception.
VOCAB.set_default_index(VOCAB['<unk>'])

Examples

In [9]:
VOCAB['yoyooyoyoy'], VOCAB['house'], VOCAB['<pad>'], VOCAB['<unk>']

(0, 324, 0, 0)

In [10]:
print(len(VOCAB))

20409


In [11]:
VOCAB.lookup_indices(TOKENIZER("House house houses ThisisnotaKNownWord"))

[324, 324, 1374, 0]

### Helper functions

In [12]:
def text_pipeline(x):
    return VOCAB(TOKENIZER(x))

Nice link on collate_fn and DataLoader in PyTorch: https://python.plainenglish.io/understanding-collate-fn-in-pytorch-f9d1742647d3

In [52]:
def collate_batch(batch):
    source_list, target_list = [], []
        
    for sentence in batch:
                        
        tokens = text_pipeline(sentence)
        
        for i in range(len(tokens)):
            if i + n -1 <= len(tokens) - 1:
                source, target = tokens[i:i+n-1], tokens[i+n-1]
                source_list.append(torch.tensor(source))
                target_list.append(torch.tensor(target))
            else:
                break
                
    source_list = torch.vstack(source_list) if source_list else torch.empty()
    target_list = torch.vstack(target_list) if target_list else torch.empty()
            
    return source_list.to(DEVICE), target_list.to(DEVICE)

### Set up the model
- nn.Embedding(V, D) is like a hash map.
- It takes in data generally of the shape N X T where N is the batch size and returns a tensor of shape N X T X D.

In [53]:
# Data is of size 16 by 10 with a vocabulary of size 10.
# Imagine that each token / word has a mapping of the sort {word -> id}.
x = torch.randint(0, 10, (16, 10))
e = nn.Embedding(10, 5)
print(x.shape)

torch.Size([16, 10])


In [54]:
e(torch.tensor(3))

tensor([ 1.5461,  1.8204,  1.9322, -0.9027,  1.2370],
       grad_fn=<EmbeddingBackward0>)

In [55]:
F.one_hot(torch.tensor(3), num_classes=10).shape

torch.Size([10])

In [56]:
F.one_hot(torch.tensor(3), num_classes=10).float() @ e.weight

tensor([ 1.5461,  1.8204,  1.9322, -0.9027,  1.2370],
       grad_fn=<SqueezeBackward3>)

In [57]:
e = nn.Embedding(10, 5)
# N X T X D - PyTorch is smart enough to realize you are passing in a batch.
print(e(x).shape)

torch.Size([16, 10, 5])


In [58]:
# One of the first Neural language models!
class NeuralLanguageModel(nn.Module):
    def __init__(self, V, m, h, n):
        super(NeuralLanguageModel, self).__init__()
        
        # Vocabulary size.
        self.V = V
        
        # Embedding dimension, per word.
        self.m = m
        
        # Hidden dimension.
        self.h = h
        
        # n in "n-gram".
        self.n = n
        
        # Can you change all this stuff to use nn.Linear?
        # Can also use nn.Parameter(torch.zeros(V, m)) for self.C but then we need one-hot and this is slow.
        self.C = nn.Embedding(V, m)
        
        # nn.Linear((n-1) * m, h, bias=False) would give the same thing for the first one below, and similarly later. 
        self.H = nn.Parameter(torch.zeros((n-1) * m, h))
        self.W = nn.Parameter(torch.zeros((n-1) * m, V))
        self.U = nn.Parameter(torch.zeros(h, V))
        
        self.b = torch.nn.Parameter(torch.ones(V))
        self.d = torch.nn.Parameter(torch.ones(h))
        
    def forward(self, x):
        
        # x is initially of dimension N X n-1 since batch is size N and context is of size n-1.
        
        # N X (n-1) X m 
        x = self.C(x)
        
        # N
        B = x.shape[0]
        
        # N X (n-1) * m
        x = x.view(B, -1)
    
        # N X V
        y = self.b + torch.matmul(x, self.W) + torch.matmul(nn.Tanh()(self.d + torch.matmul(x, self.H)), self.U)
        
        return y

### Set up the 

In [59]:
criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
model = NeuralLanguageModel(len(VOCAB), m, h, n).to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

### Set up the data

In [60]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")
test_iter = DATASETS[DATASET](root=DATA_DIR, split="test")

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

### Train the model

In [61]:
def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_loss, total_batches = 0.0, 0.0
    log_interval = 100

    for idx, (x, y) in tqdm(enumerate(dataloader)):
        optimizer.zero_grad()
        
        if x.nelement() == 0:
            continue
        
        logits = model(x)
                        
        # Get the loss.
        loss = criterion(input=logits, target=y.squeeze(-1))

        # Do back propagation.
        loss.backward()
                        
        # Clip the gradients.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        
        # Do an optimization step.
        optimizer.step()
        total_loss += loss.item()
        total_batches += 1
                
        if idx % log_interval == 0 and idx > 0:
            perplexity = torch.exp(torch.tensor(total_loss / total_batches)).item()
            print(
                "| epoch {:3d} "
                "| {:5d}/{:5d} batches "
                "| perplexity {:8.3f} "
                "| loss {:8.3f} "
                .format(
                    epoch,
                    idx,
                    len(dataloader),
                    perplexity,
                    total_loss / total_batches,
                )
            )
            total_loss, total_batches = 0.0, 0

In [62]:
def evaluate(dataloader, model, criterion):
    model.eval()
    total_loss, total_batches = 0.0, 0

    with torch.no_grad():
        for idx, (x, y) in enumerate(dataloader):
            logits = model(x)
            total_loss += criterion(input=logits, target=y.squeeze(-1)).item()
            total_batches += 1
    return total_loss / total_batches, torch.exp(torch.tensor(total_loss / total_batches)).item()

In [63]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, criterion, epoch)
    loss_val, perplexity_val = evaluate(valid_dataloader, model, criterion)
    scheduler.step()
    print("-" * 59)
    print(
        "| end of epoch {:3d} "
        "| time: {:5.2f}s "
        "| valid perplexity {:8.3f} "
        "| valid loss {:8.3f}".format(
            epoch,
            time.time() - epoch_start_time,
            perplexity_val,
            loss_val
        )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
loss_test, perplexity_test = evaluate(test_dataloader, model, criterion)
print("test perplexity {:8.3f} | test loss {:8.3f} ".format(perplexity_test, loss_test))

103it [00:09, 11.51it/s]

| epoch   1 |   100/ 2181 batches | perplexity 1365.428 | loss    7.219 


202it [00:18, 10.86it/s]

| epoch   1 |   200/ 2181 batches | perplexity  794.575 | loss    6.678 


302it [00:27, 11.02it/s]

...

| epoch   5 |  2000/ 2181 batches | perplexity  390.331 | loss    5.967 


2102it [03:08, 11.59it/s]

| epoch   5 |  2100/ 2181 batches | perplexity  385.240 | loss    5.954 


2181it [03:16, 11.12it/s]


-----------------------------------------------------------
| end of epoch   5 | time: 201.36s | valid perplexity  420.861 | valid loss    6.042
-----------------------------------------------------------
Checking the results of test dataset.
test perplexity  338.582 | test loss    5.825 


## Questions:
- What is wrong with this implementation?
- Preprocess! ... Or Normalize Correctly!
- My batches are sentences, and a sentence might give rise to many (x, y) pairs.
- Each sentence - batch has 16 sentences.
- I.e. if I have batch 1 maybe it gives rise to 200 (x, y) pairs and another gives rise to 100 (x, y) pairs.
- Assume the sum of the cross-etropy is $L_1$ for the first batch and $L_2$ for the second batch.
- If I have two batches, one of size 200 and another of size 100, I get ($L_1$/200 + $L_2$/100) / 2 != ($L_1$ + $L_2$)(200 + 100)
- Can you fix this?
- HW!